# Some Tips for JupyterNotebook : 

-Scroll up and down your cells with your Up and Down keys.

-Press A or B to insert a new cell above or below the active cell.

-M will transform the active cell to a Markdown cell.

-Y will set the active cell to a code cell.

-D + D (D twice) will delete the active cell.

-Z will undo cell deletion.

-Hold Shift and press Up or Down to select multiple cells at once. With multiple cells selected, Shift + M will merge your selection.

# Some tests for text and running code 


In [1]:
print("hello world !")

hello world !


In [2]:
x = 3
for i in range(0, 5):
    print(i + x) 

3
4
5
6
7


Salut text intermediaire 

In [3]:
import time 
time.sleep(3)

Salut 

Je tape ce texte pour tester le Markdown 

Ca marche bien !

In [4]:
string = "Bonjour comment ca va ? "
print(string)

Bonjour comment ca va ? 


# Algorithm(s) used for the AI part of the app 


In [5]:
max_score = 10
min_score = 0
critical = 0

def input_score(string, list_input):
    score = int(input("Rentrez votre " + string + "\t"))
    list_input.append(score)
    return score

# max_score_is_bad is a boolean
def find_factor(score,name, epsilon, max_score_is_bad):
    if (max_score_is_bad): 
        if (abs(max_score - score) < epsilon):
            print(name + " est un facteur important du modele")
    else :
        if (score < epsilon):
            print(name + " est un facteur important du modele")


input_list = []
age     = input_score("age", input_list)
fatigue = input_score("fatigue", input_list)
douleur = input_score("douleur", input_list)
sommeil = input_score("sommeil", input_list)

del input_list[0] # we don t consider the age as an input for this time 
    
print(input_list)

# 1st exemple 
if ((max_score - fatigue) < 2 or sommeil < 2 or (max_score - douleur) < 2):
    critical = 1
    print("You should try to do this or that ")
elif (fatigue > 2 and fatigue < 5):
    print("Keep going it s better")
else : 
    print("Great job you seems to be good ! ")
    
# 2nd exemple 
for score in input_list:
    if score < 2:
        critical = 1
        print("You should try to do this or that #2")
        
        
        
# do the correlation between the age and the (back) pain for example
if (douleur > 7):
    if (age > 60):
        print("Try to stretch yourself ")
    else:
        print("Change your position when you are working or whatever")

find_factor(fatigue, "fatigue", 2, 1)
find_factor(douleur, "douleur", 2, 1)
find_factor(sommeil, "sommeil", 2, 0)

Rentrez votre age	45
Rentrez votre fatigue	2
Rentrez votre douleur	3
Rentrez votre sommeil	2
[2, 3, 2]
Great job you seems to be good ! 


# TEST 